In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [2]:
FEATURES = [
    "gd",
    "ed",
    "gp",
    "isr",
#    "views", 
    "large", 
    "top", 
#    "speed_level",
    "speed_level 1",
    "speed_level 2",
    "speed_level 3",
    "price"]

In [3]:
def get_speed_level(speed_level):
    if speed_level == 1:
        return  [1]
        #return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        return [0.4]
        #return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        return [0.1]
        #return [0,0,1]
        #print X[-1], speed_level
    else:
        return [0]
        #return [0,0,0]
        #print X[-1], speed_level

In [141]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        
        #gd
        gd = [0.060, 0.223, 0.015, 0.039, -0.003, 0.011, 8.370] #unnormed
        gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
        #gd = [1, 1, 1, 1, 1, 1, 0]
        x = [
            doc['degree'].count(u'Профессор'),
            int(doc['top']),
            len(doc['illnesses']),
            len(doc['languages']),
            int(doc['h_index']),
            int(doc['experience']),
            1
        ]
        X[-1]+= [round(sum(np.array(gd)*np.array(x)),5)] 
        #ed
        ed = [-0.552, -0.032, 0.533, 0.098, 8.945] #unnormed
        ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
        #ed = [1, 1, 1, 1, 0]
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        x = [
            len(list(set(ids_diad))),
            len(doc['procedures']['diagnostics']),
            len(list(set(ids_oper))),
            len(doc['procedures']['operations']),
            1
        ]
        X[-1]+= [round(sum(np.array(ed)*np.array(x)),5)]
        #gp
        gp = [-1.547, -0.204, 1.837, 0.139] #unnormed
        gp = [-3.094, -7.545, 11.025, 9.011]
        #gp = [1, 1, 1, 1]
        x = [
            sum([int(i['top']) for i in all_proc]), #top_proc_cli
            len(all_proc), #gen_proc_cli
            sum([int(i['top']) for i in clinic['procedures']['operations']]), #top_oper_cli
            len(clinic['procedures']['operations']), #gen_oper_cli
        ]
        X[-1]+= [round(sum(np.array(gp)*np.array(x)),5)]
        print X[-1]
        print x
        print
        #
        c_id = int(clinic['country_id'])
        if c_id == 1:
            X[-1]+=[1]
        else:
            X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[get_speed_level(int(clinic['speed_level']))]
        X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        
    y=[int(clinic_estim['rating'])] * len(all_doc)
    return [X, y]

In [1]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [130]:
def normale(mat):
    for i in [0,2]:
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [8]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X, y = get_X_y_from(cancer_data)

In [2]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

#normale(X_all)

NameError: name 'json' is not defined

In [67]:
estims_data[3]

{u'experience': u'8',
 u'id': u'78',
 u'name': u'\u0422\u0435\u043a\u043d\u043e\u043d',
 u'price': u'4',
 u'rating': u'7',
 u'specialist': u'8',
 u'variety': u'10'}

In [66]:
len(cancer_data[3]['doctors'])

1

In [140]:
X_all

[[18.422, 9.327, -150.438, 1, 1, 1, 0.5],
 [9.769, 10.431, -150.438, 1, 1, 1, 0.5],
 [31.246, 4.971, -150.438, 1, 1, 1, 0.5],
 [14.233, 9.471, -150.438, 1, 1, 1, 0.5],
 [16.008, 7.995, -150.438, 1, 1, 1, 0.5],
 [13.476, 10.143, -150.438, 1, 1, 1, 0.5],
 [9.114, 2.091, -175.746, 0, 0, 0, 0.4],
 [7.284, 7.179, -213.418, 0, 1, 0, 0.3],
 [24.972, 12.351, -147.453, 1, 1, 1, 0.7],
 [22.098, 7.179, -147.453, 1, 1, 1, 0.7],
 [2.127, 2.235, -93.958, 0, 0, 0, 0.6],
 [8.802, 1.191, -126.455, 0, 1, 0, 0.3],
 [12.732, 3.051, -94.388, 0, 0, 0, 0.1],
 [24.972, 12.351, -147.453, 1, 1, 1, 0.6],
 [19.764, 9.471, -147.453, 1, 1, 1, 0.6],
 [16.992, 11.391, -147.453, 1, 1, 1, 0.6],
 [18.114, 9.327, -147.453, 1, 1, 1, 0.6],
 [16.05, 3.195, -213.418, 0, 1, 0, 0.3],
 [7.857, 7.179, -213.418, 0, 1, 0, 0.3],
 [8.67, 1.191, -213.418, 0, 1, 0, 0.3],
 [18.422, 9.327, -150.438, 1, 1, 1, 0.4],
 [9.769, 10.431, -150.438, 1, 1, 1, 0.4],
 [31.246, 4.971, -150.438, 1, 1, 1, 0.4],
 [8.802, 9.471, -150.438, 1, 1, 1, 0.4],

In [ ]:
for i in range(1,10):
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

In [134]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

Total: 64, train: 57, test: 7
Residual sum of squares: 5.83
Train absolute: 1.96
Test absolute: 1.90
Absolute to mean: 29.63%
Train variance score: 0.39
Test variance score: 0.11


In [138]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_4)):    
    print "Real: %f \t Predicted: %f" %(y_4[i], regr.predict(X_4[i]))
    print X_4[i]
    print
    #print y_5[i]
    #print regr.predict(X_5[i])[0]

Real: 10.000000 	 Predicted: 7.807188
[1.0, 4.971, -0.7049, 1, 1, 1, 0.5]

Real: 10.000000 	 Predicted: 7.482312
[0.4555, 9.471, -0.7049, 1, 1, 1, 0.5]

Real: 10.000000 	 Predicted: 7.448450
[0.3155, 9.471, -0.7049, 1, 1, 1, 0.5]

Real: 10.000000 	 Predicted: 7.787534
[0.6992, 3.734, -0.7049, 1, 1, 1, 0.5]

Real: 9.000000 	 Predicted: 3.675460
[0.5285, -1.294, -0.3974, 0, 0, 0, 0.2]

Real: 8.000000 	 Predicted: 6.830363
[0.5137, 3.195, -1.0, 0, 1, 0, 0.2]

Real: 8.000000 	 Predicted: 7.047286
[0.4541, -2.194, -1.0, 0, 1, 0, 0.2]

Real: 7.000000 	 Predicted: 8.035695
[0.6888, -5.218, -0.8235, 0, 0, 0, 0.6]

Real: 6.000000 	 Predicted: 0.810060
[0.2798, -4.258, -0.3904, 0, 1, 0, 0.4]

Real: 5.000000 	 Predicted: 4.320903
[0.4727, -4.258, -0.7182, 0, 1, 0, 0.1]

Real: 4.000000 	 Predicted: 4.440375
[0.7161, -5.362, -0.4423, 0, 0, 0, 0.0]

Real: 3.000000 	 Predicted: 3.553173
[0.5363, -4.258, -0.3901, 0, 0, 0, 0.6]

Real: 2.000000 	 Predicted: 7.255230
[0.7992, 12.351, -0.6909, 1, 1, 1, 0.

In [137]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0] #, i[1]

Features sorted by their score:
-0.457882626999
10
0.2419
-0.0429
-10.1625
2.0337
-2.7727
2.0337
-0.4431


In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
plt.plot([i[22] for i in X])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)